In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('datasets/df_abt.csv')
df.head()

,cntry,sclmeet,inprdsc,aesfdrk,health,hlthhmp,sclact,crmvct,fltsd,fltdpr,...,sedirlf,wrbknrm,uempla,uempli,Sadness,Enthusiasm,Happiness,Optimism,Trust,Helpful
0,Albania,Once a week,3,Very safe,Fair,No,About the same,Less than most,Some of the time,Most of the time,...,3,Disagree,0,0,Some of the time,8,7,Agree,2,3
1,Albania,Once a month,1,Unsafe,Good,No,Less than most,Less than most,All or almost all of the time,All or almost all of the time,...,10,Agree,1,0,Most of the time,6,0,Agree strongly,0,3
2,Albania,Every day,10 or more,Unsafe,Very good,No,About the same,Much less than most,Some of the time,Some of the time,...,10,Disagree strongly,0,0,Some of the time,8,5,Agree strongly,4,3
3,Albania,Several times a week,2,Very safe,Fair,No,About the same,Less than most,Most of the time,Some of the time,...,88,Agree strongly,0,0,Some of the time,99,2,Agree,6,9
4,Albania,Several times a month,None,Safe,Fair,No,Less than most,Less than most,Most of the time,Some of the time,...,10,Agree,0,0,Some of the time,9,6,Agree,6,5


## Export Scatter Plot Data

In [8]:
df_trstprl_happiness = df[["trstprl", "Happiness"]]
df_trstprl_happiness.to_csv('datasets/df_trstprl_happiness.csv', index=False)

## Create dataset for arc plot

In [9]:
group_mapping = {
    "trstplt": 0,
    "trstprl": 1,
    "trstlgl": 2,
    "trstplc": 3,
    "Happiness": 4,
    "trstprt": 5,
    "trstep": 6,
    "trstun": 7
}

In [10]:
ess = {}
ess["nodes"] = []
for k, v in group_mapping.items():
    for i in range(11):
        ess["nodes"].append({"name": f"{k}_{i}", "group": v, "index": v*11+i})

In [11]:
columns_to_filter = group_mapping.keys()
cleaned_df = df.copy()
for col in columns_to_filter:
    cleaned_df = cleaned_df[cleaned_df[col] < 11]

In [12]:
from collections import defaultdict

links = defaultdict(int)
trst_cols = [col for col in columns_to_filter if col != "Happiness"]
for i, row in cleaned_df.iterrows():
    for col in trst_cols:
        links[f"{col}_{row[col]}_{row['Happiness']}"] += 1 

In [13]:
node_to_index_mapping = {node["name"]: node["index"] for node in ess["nodes"]}
happiness_val_index_mapping = {f"Happiness_{i}": 44+i for i in range(11)}

In [14]:
import random
# format links for vega
formatted_links = []
for k, v in links.items():
    trst_col, trust_val, happiness_val = k.split("_")
    formatted_links.append({'source': node_to_index_mapping[f"{trst_col}_{trust_val}"], 
                            'target': node_to_index_mapping[f"Happiness_{happiness_val}"], 
                            'value': v/100})

ess["links"] = formatted_links

In [16]:
with open("./datasets/arc_plot_trst_vs_happiness.json", "w+") as f:
    json.dump(ess, f)

## Create Undirected Dataset

In [17]:
ess_network = {}
ess_network["nodes"] = []
for k, v in group_mapping.items():
    for i in range(11):
        ess_network["nodes"].append({"name": f"{k}_{i}", "group": v, "index": v*11+i})

In [18]:
links = defaultdict(int)
trst_cols = [col for col in columns_to_filter if col != "Happiness"]
for _, row in cleaned_df.iterrows():
    for i, col_i in enumerate(columns_to_filter):
        for j, col_j in enumerate(columns_to_filter):
            if j > i:
                links[f"{col_i}_{row[col_i]}__{col_j}_{row[col_j]}"] += 1 
                links[f"{col_j}_{row[col_j]}__{col_i}_{row[col_i]}"] += 1 

for link in links:
    links[link] /= 2

In [19]:
formatted_links = []
for k, v in links.items():
    source, target = k.split("__")
    if v > 75:  # This reveals some interpretation
        formatted_links.append({'source': node_to_index_mapping[source], 'target': node_to_index_mapping[target], 'value': v})
    
ess_network["links"] = formatted_links

with open("./datasets/network_plot_undirected_dataset.json", "w+") as f:
    json.dump(ess_network, f)